In [39]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
import numpy as np
#from sklearn.pipeline import Pipeline
from arch import arch_model

In [40]:
def load_data(symbol="SPY", start="1994-01-01", end="2021-01-01", interval='1d'):
    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)
    return df

def preprocess_data(df):
    df = df[['Date', 'Close']]
    df.dropna(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def train_test(list_df, train_size=0.7, test_size=0.3):
    result = []
    for df in list_df:
        X = df.index.values.reshape(-1, 1)  
        y = df['Close'].values              

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=train_size, test_size=test_size, shuffle=False
        )
        result.append([X_train, X_test, y_train, y_test])
    return result

def etl_pipeline(train_size=0.7, test_size=0.3):
    gspc = load_data('^GSPC')
    dji = load_data('^DJI')
    nyse = load_data('^NYA')
    
    gspc_close = preprocess_data(gspc)
    dji_close = preprocess_data(dji)
    nyse_close = preprocess_data(nyse)

    list_df_close = [gspc_close, dji_close, nyse_close]

    splits = train_test(list_df_close, train_size=train_size, test_size=test_size)

    return {
        "GSPC": {"X_train": splits[0][0], "X_test": splits[0][1], "y_train": splits[0][2], "y_test": splits[0][3]},
        "DJI":  {"X_train": splits[1][0], "X_test": splits[1][1], "y_train": splits[1][2], "y_test": splits[1][3]},
        "NYSE": {"X_train": splits[2][0], "X_test": splits[2][1], "y_train": splits[2][2], "y_test": splits[2][3]},
    }



In [41]:
gspc=load_data('^GSPC')
dji=load_data('^DJI')
nyse=load_data('^NYA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [42]:
gspc_close=preprocess_data(gspc)
dji_close=preprocess_data(dji)
nyse_close=preprocess_data(nyse)

C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [43]:
print(gspc_close)

Price             Close
Ticker            ^GSPC
Date                   
1994-01-03   465.440002
1994-01-04   466.890015
1994-01-05   467.549988
1994-01-06   467.119995
1994-01-07   469.899994
...                 ...
2020-12-24  3703.060059
2020-12-28  3735.360107
2020-12-29  3727.040039
2020-12-30  3732.040039
2020-12-31  3756.070068

[6799 rows x 1 columns]


In [44]:
# Run the ETL pipeline
result = etl_pipeline()

# Assign GSPC values
X_gspc_train = result["GSPC"]["X_train"]
X_gspc_test  = result["GSPC"]["X_test"]
y_gspc_train = result["GSPC"]["y_train"]
y_gspc_test  = result["GSPC"]["y_test"]

# Assign DJI values
X_dji_train = result["DJI"]["X_train"]
X_dji_test  = result["DJI"]["X_test"]
y_dji_train = result["DJI"]["y_train"]
y_dji_test  = result["DJI"]["y_test"]

# Assign NYSE values
X_nyse_train = result["NYSE"]["X_train"]
X_nyse_test  = result["NYSE"]["X_test"]
y_nyse_train = result["NYSE"]["y_train"]
y_nyse_test  = result["NYSE"]["y_test"]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
C:\Users\idris\AppData\Local\Temp\ipykernel_2892\3814708522.py:8: SettingW

In [45]:
print(X_dji_train)

[['1994-01-03T00:00:00.000000000']
 ['1994-01-04T00:00:00.000000000']
 ['1994-01-05T00:00:00.000000000']
 ...
 ['2012-11-20T00:00:00.000000000']
 ['2012-11-21T00:00:00.000000000']
 ['2012-11-23T00:00:00.000000000']]


In [46]:


# Function to calculate log returns
def calculate_log_returns(prices):
    return np.log(prices / prices.shift(1) + 1e-8).dropna()

# Modified GARCH(1,1) prediction function
def garch_predictions(prices_series, forecast_horizon=1):
    mean_predictions = []
    volatility_predictions = []
    prediction_dates = []

    prices_series = prices_series.sort_index()

    for i in range(90, len(prices_series)):
        window_prices = prices_series.iloc[i-90:i]
        log_returns = calculate_log_returns(window_prices)

        model = arch_model(log_returns, mean='constant', vol='GARCH', p=1, q=1,rescale=True)
        model_fit = model.fit(disp='off')

        forecast = model_fit.forecast(horizon=forecast_horizon)

        predicted_mean = forecast.mean.iloc[-1].values[0]
        predicted_volatility = np.sqrt(forecast.variance.iloc[-1].values[0])

        mean_predictions.append(predicted_mean)
        volatility_predictions.append(predicted_volatility)
        prediction_dates.append(prices_series.index[i])

    true_volatility = (calculate_log_returns(prices_series.iloc[90:]) - pd.Series(mean_predictions, index=prediction_dates)) ** 2

    predictions_df = pd.DataFrame({
        'Date': prediction_dates,
        'Predicted_Mean': mean_predictions,
        'Predicted_Volatility': volatility_predictions,
        'True_Volatility': true_volatility
    }).set_index('Date')

    return predictions_df



In [47]:
# LSTM Model Definition
def create_lstm_model():
    model = models.Sequential([
        layers.LSTM(256, return_sequences=True, input_shape=(90, 1)),
        layers.Dropout(0.2),
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(1)
    ])
    return model

# Custom Loss Function
def custom_loss(lambda_param):
    def loss(y_true, y_pred):
        predicted_variance_garch = y_true[:, 1]  # Second column is GARCH predictions
        true_variance = y_true[:, 0]  # First column is ground truth variance
        mse = tf.keras.losses.MeanSquaredError()
        return (lambda_param * mse(true_variance, y_pred) + 
                (1 - lambda_param) * mse(predicted_variance_garch, y_pred))
    return loss




In [48]:
# Example Usage
prices_series = pd.Series(np.random.randn(500), index=pd.date_range(start='2020-01-01', periods=500))
predictions_df = garch_predictions(prices_series)

true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparing the data for LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))]).reshape(-1, 90, 1)
y = np.vstack((true_volatility[90:], predicted_volatility[90:]))  # Ground truth and GARCH predictions

# Model Initialization and Compilation
model = create_lstm_model()
model.compile(optimizer='adam', loss=custom_loss(lambda_param=0.5))

# Model Training
model.fit(X, y.T, epochs=20, batch_size=32, shuffle=True)

c:\Users\idris\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\idris\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\idris\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\idris\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\idris\anaconda3\envs\tensorflow_env\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\idris\anaconda3\

Epoch 1/20
10/10 [==============================] - 9s 260ms/step - loss: nan
Epoch 2/20
10/10 [==============================] - 3s 267ms/step - loss: nan
Epoch 3/20
10/10 [==============================] - 3s 265ms/step - loss: nan
Epoch 4/20
10/10 [==============================] - 3s 291ms/step - loss: nan
Epoch 5/20
10/10 [==============================] - 3s 266ms/step - loss: nan
Epoch 6/20
10/10 [==============================] - 3s 295ms/step - loss: nan
Epoch 7/20
10/10 [==============================] - 3s 271ms/step - loss: nan
Epoch 8/20
10/10 [==============================] - 3s 307ms/step - loss: nan
Epoch 9/20
10/10 [==============================] - 3s 289ms/step - loss: nan
Epoch 10/20
10/10 [==============================] - 3s 268ms/step - loss: nan
Epoch 11/20
10/10 [==============================] - 3s 279ms/step - loss: nan
Epoch 12/20
10/10 [==============================] - 3s 298ms/step - loss: nan
Epoch 13/20
10/10 [==============================] - 3s 278ms